In [1]:
!pip install langchain huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
!pip install transformers sentence_transformers accelerate bitsandbytes

In [10]:
from getpass import getpass
HUGGINGFACE_API_TOKEN=getpass()

··········


In [11]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]=HUGGINGFACE_API_TOKEN

In [12]:
from langchain_community.llms import HuggingFaceHub

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [13]:
# test the model
question="Where is the capital of China?"
template=""" Question: {question} Answer:let's think step by step."""

prompt=PromptTemplate(template=template,input_variables=['question'])

In [14]:
repo_id='google/flan-t5-base'

In [15]:
llm=HuggingFaceHub(
    repo_id=repo_id
)
llm_chain=LLMChain(prompt=prompt,llm=llm,llm_kwargs={"temperature":0,"max_length":512})
print(llm_chain.run(question))

China is a country located in the north of China. The capital of China is Beijing. The answer: Beijing.


RAG search

In [16]:
from langchain.document_loaders.csv_loader import CSVLoader
#from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS

In [17]:
from langchain.document_loaders import PyPDFLoader

In [18]:
import pandas as pd

loader = CSVLoader(file_path="new_processed_file.csv")
pages= loader.load()


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter= RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50,)
docs=text_splitter.split_documents(pages)

In [19]:
len(docs)
type(docs[0])

langchain_core.documents.base.Document

In [ ]:
# check whether the docs is empty
if not docs:
    print("the splitted file is empty")
else:
    print(f"generate {len(docs)} files")

print(f"Loaded {len(pages)} pages.")
print(f"Split into {len(docs)} documents.")

doc = docs[0]
print(dir(doc))
print(doc)
print(docs)

os.environ["HUGGINGFACE_API_TOKEN"] = HUGGINGFACE_API_TOKEN

dir(docs)

In [22]:
print(type(docs[0]))
print(dir(docs[0]))
texts = [doc.page_content for doc in docs]

<class 'langchain_core.documents.base.Document'>
['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_enforce_dict

In [23]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings=HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGINGFACE_API_TOKEN, model_name='BAAI/bge-base-en-v1.5'
)

In [ ]:
# test whether the embeddings work
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result

In [32]:
!pip install faiss-gpu # this package is required to use the FAISS

## Create a FAISS database from documents and their embeddings.
# This database will be used for similarity searches.
db=FAISS.from_documents(docs,embeddings)

# Define a query to search within the documents.
query='Who is the author of Chani is the real protagonist of Dune: Part Two?'

In [33]:
# Print the page content of the first document for inspection purposes.
print(docs[0].page_content)

author: 
title: Forian GAAP EPS of $0.04 beats by $0.07, revenue of $5.37M misses by $0.19M
description: Forian GAAP EPS of $0.04 beats by $0.07, revenue of $5.37M misses by $0.19M
source: Seeking Alpha
category: business
date: 2024-03-28T20:26:56+00:00
release_time: 2024-03-28T20:26:56


In [34]:
# Perform a similarity search in the FAISS database with the specified query
# and retrieving the top 3 similar items.
result_simi = db.similarity_search(query , k = 3)

In [35]:
result_simi

[Document(page_content='author: Austen Goslin\ntitle: Chani is the real protagonist of Dune: Part Two\ndescription: Photo: Chiabella James/Warner Bros. Pictures This is Zendaya’s movie, and the storytelling choices reflect it Continue reading&hellip;\nsource: Polygon\ncategory: entertainment\ndate: 2024-03-28T20:16:16+00:00', metadata={'source': 'new_processed_file.csv', 'row': 18}),
 Document(page_content='author: Jeffrey Parkin\ntitle: ‘Readvent of Calamity’ quest walkthrough in Dragon’s Dogma 2\ndescription: Image: Capcom via Polygon Where to find Ulrika in Melve ‘from time to time’ Continue reading&hellip;\nsource: Polygon\ncategory: entertainment\ndate: 2024-03-26T20:57:25+00:00', metadata={'source': 'new_processed_file.csv', 'row': 65}),
 Document(page_content='author: Akashdeep Banerjee\ntitle: Unveiling 76ers’ ‘Worst Kept Secret’: LeBron James, Kawhi Leonard, and Paul George Saga', metadata={'source': 'new_processed_file.csv', 'row': 45})]

In [36]:
# Aggregate the page contents of the top 3 similar items into a single string, separated by new lines.
# This aggregated content serves as the source knowledge context for answering the query.
source_knowledge = "\n".join([x.page_content for x in result_simi])

# Construct an augmented prompt that includes both the source knowledge context and the query.
# This prompt is structured to facilitate the generation of an answer based on the provided context.
augmented_prompt = """Using the contexts below, answer the query.

contexts:
{source_knowledge}

query: {query}"""


In [37]:
prompt = PromptTemplate(template=augmented_prompt, input_variables=["source_knowledge" ,"query"])

llm_chain = LLMChain(prompt=prompt, llm=llm  , llm_kwargs = {"temperature":0, "max_length":1024})

print(llm_chain.run( {"source_knowledge":source_knowledge ,"query" : query }))


Austen Goslin


In [59]:
query2="Linda Bean died at the age of ?"
result_simi2 = db.similarity_search(query2 , k = 5)
source_knowledge2 = "\n".join([x.page_content for x in result_simi2])
augmented_prompt2 = """Using the contexts below, answer the query.
contexts:
{source_knowledge2}

query: {query2}"""

prompt2 = PromptTemplate(template=augmented_prompt2, input_variables=["source_knowledge2" ,"query2"])
llm_chain2 = LLMChain(prompt=prompt2, llm=llm  , llm_kwargs = {"temperature":0.6, "max_length":1024})
print(llm_chain2.run( {"source_knowledge2":source_knowledge2 ,"query2" : query2 }))

82
